In [46]:
import arcpy
import os
from arcpy import env

### Workspace and geodatabase
proj_gdb = r"" # replace with your project gdb
ws = arcpy.env.workspace = proj_gdb
arcpy.env.overwriteOutput = True

###Declare variables for the feature classes
hunting_fc = "" # replace with your hunting parcels feature class
roads_fc = "" # replace with your roads feature class
structs_fc = "" # replace with your buildings feature class

### Declare layer variables for the proximate roads and buffers
roads_lyr = 'roads_lyr'
structs_lyr = 'structs_lyr'
hunting_proximate_roads_lyr = "Hunting_proximate_roads_lyr"
hunting_proximate_structs_lyr = "Hunting_proximate_structs_lyr"

### Select roads and structures around hunting parcels - 2000 ft is adjustable
### This step reduces the size of the roads and structures layers, and so improves speed
def getSurroundingDataByLocation():
    select_distance = "2000 Feet"
    arcpy.management.MakeFeatureLayer(roads_fc, roads_lyr)
    arcpy.management.SelectLayerByLocation(roads_lyr, "WITHIN_A_DISTANCE", hunting_fc, select_distance)
    arcpy.management.CopyFeatures(roads_lyr, hunting_proximate_roads_lyr)
    
    arcpy.management.MakeFeatureLayer(structs_fc, structs_lyr)
    arcpy.management.SelectLayerByLocation(structs_lyr, "WITHIN_A_DISTANCE", hunting_fc, select_distance)
    arcpy.management.CopyFeatures(structs_lyr, hunting_proximate_structs_lyr)

getSurroundingDataByLocation()

road_buf = hunting_proximate_roads_lyr + "_buffer"
struct_buf = hunting_proximate_structs_lyr + "_buffer"
merged_buf = "All_Buffers"
merged_buf_dissolved = "All_Buffers_Dissolved"
output_fc = "Hunting_Buffer"

### Create and merge buffers
def createBuffers():
    arcpy.analysis.Buffer(hunting_proximate_roads_lyr, road_buf, "150 Feet", "FULL", "ROUND", "ALL")
    arcpy.analysis.Buffer(hunting_proximate_structs_lyr, struct_buf, "500 Feet", "FULL", "ROUND", "ALL")
    arcpy.management.Merge([road_buf, struct_buf], merged_buf_dissolved)
    
createBuffers()

# Field maps set up - not currently in use
fm_type = arcpy.FieldMap()
fm_diam = arcpy.FieldMap()
fms = arcpy.FieldMappings()

def intersectBuffersAndParcels():
    arcpy.analysis.Intersect([hunting_fc, merged_buf_dissolved], output_fc, "NO_FID")
       # arcpy.management.Dissolve(merged_buf_dissolved, merged_buf, "Name", fms)
    
intersectBuffersAndParcels()

# Deletes temporary layers created during previous geoprocessing tasks
def cleanup():
    arcpy.management.Delete([roads_lyr, structs_lyr, hunting_proximate_roads_lyr,
                             hunting_proximate_structs_lyr, road_buf, struct_buf,
                             merged_buf, merged_buf_dissolved])
    
cleanup()

# Update the buffer feature class symbology
def updateSymbology(layerName):
    activeMap = arcpy.mp.ArcGISProject("CURRENT").activeMap
    layers = activeMap.listLayers()
    currLyr = layers[0]
    for lyr in layers:
        if lyr == layerName:
            currLyr = lyr
    
    sym = currLyr.symbology
    pink = {"RGB": [255, 127, 127, 100]}
    if currLyr.isFeatureLayer and hasattr(sym, "renderer"):
        # Customize symbol to have hatched line and color pink
        sym.renderer.symbol.applySymbolFromGallery("Hatch line")
        sym.renderer.symbol.color = pink
        sym.renderer.symbol.outlineWidth = 1
        sym.renderer.symbol.outlineColor = pink

        currLyr.symbology = sym

updateSymbology(output_fc)

# replace with path to output csv file
csvfile = r""
def exportAttributesToCsv():
    with open(csvfile, "w") as output:
        writer = csv.writer(output, lineterminator='\n')

        #TODO - calculate Buf_Acres geometry in earlier function, has to be manually added to Attribute Table right now
        fields = ["Name", "Town", "Acres", "Buf_Acres"]
        with arcpy.da.SearchCursor(output_fc, fields) as cursor:
            for row in cursor:
                writer.writerow(row)
    
exportAttributesToCsv()